In [200]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
## 设置字体
from matplotlib.font_manager import FontProperties
fonts = FontProperties(fname = "/Library/Fonts/华文细黑.ttf",size=14)
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [289]:
import pandas as pd
train = pd.read_csv("./res_train")

In [319]:
import jieba
train = train.dropna()
tokenizer = jieba.Tokenizer()
train['tokens'] = train.content.apply(lambda x: list(tokenizer.cut(str(x))))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\瓜瓜\AppData\Local\Temp\jieba.cache
Loading model cost 0.568 seconds.
Prefix dict has been built successfully.


In [320]:
train.head()

,content,label,tokens
0,什么是电气一次设备和一次回路,1.0,"[什么, 是, 电气, 一次, 设备, 和, 一次, 回路]"
1,一次设备是指直接生产输送和分配电能的高压电气设备它包括发电机变压器断路器隔离开关自动开关接触...,0.0,"[一次, 设备, 是, 指, 直接, 生产, 输送, 和, 分配, 电能, 的, 高压, 电..."
2,什么是电气二次设备和二次回路,1.0,"[什么, 是, 电气, 二次, 设备, 和, 二次, 回路]"
3,二次设备是指对一次设备的工作进行监测控制调节保护以及为运行维护人员提供运行工况或生产指挥信号...,0.0,"[二次, 设备, 是, 指, 对, 一次, 设备, 的, 工作, 进行, 监测, 控制, 调..."
4,哪些回路属于连接保护装置的二次回路,1.0,"[哪些, 回路, 属于, 连接, 保护装置, 的, 二次, 回路]"


In [321]:
train.tokens = train.tokens.apply(lambda x: ' '.join(x)).tolist()
train_y = train.label.astype("int").values
train.head()

,content,label,tokens
0,什么是电气一次设备和一次回路,1.0,什么 是 电气 一次 设备 和 一次 回路
1,一次设备是指直接生产输送和分配电能的高压电气设备它包括发电机变压器断路器隔离开关自动开关接触...,0.0,一次 设备 是 指 直接 生产 输送 和 分配 电能 的 高压 电气设备 它 包括 发电机 ...
2,什么是电气二次设备和二次回路,1.0,什么 是 电气 二次 设备 和 二次 回路
3,二次设备是指对一次设备的工作进行监测控制调节保护以及为运行维护人员提供运行工况或生产指挥信号...,0.0,二次 设备 是 指 对 一次 设备 的 工作 进行 监测 控制 调节 保护 以及 为 运行 ...
4,哪些回路属于连接保护装置的二次回路,1.0,哪些 回路 属于 连接 保护装置 的 二次 回路


In [323]:
le = LabelEncoder()
train_y = le.fit_transform(train.label).reshape(-1,1)
ohe = OneHotEncoder()
train_y = ohe.fit_transform(train_y).toarray()
train_y

array([[0., 1.],
       [1., 0.],
       [0., 1.],
       ...,
       [0., 1.],
       [1., 0.],
       [1., 0.]])

In [310]:
## 使用Tokenizer对词组进行编码
## 当我们创建了一个Tokenizer对象后，使用该对象的fit_on_texts()函数，以空格去识别每个词,
## 可以将输入的文本中的每个词编号，编号是根据词频的，词频越大，编号越小。
max_words = 5000
max_len = 30
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(train.tokens)

## 使用word_index属性可以看到每次词对应的编码
## 使用word_counts属性可以看到每个词对应的频数
for ii,iterm in enumerate(tok.word_index.items()):
    if ii < 10:
        print(iterm)
    else:
        break
print("= = = = = =")
for ii,iterm in enumerate(tok.word_counts.items()):
    if ii < 10:
        print(iterm)
    else:
        break

('的', 1)
('树脂', 2)
('是', 3)
('在', 4)
('和', 5)
('再生', 6)
('有', 7)
('水中', 8)
('处理', 9)
('交换', 10)
= = = = = =
('什么', 313)
('是', 663)
('电气', 11)
('一次', 22)
('设备', 64)
('和', 508)
('回路', 59)
('指', 21)
('直接', 17)
('生产', 49)


In [311]:
## 对每个词编码之后，每句新闻中的每个词就可以用对应的编码表示，即每条新闻可以转变成一个向量了：
train_seq = tok.texts_to_sequences(train.tokens)
## 将每个序列调整为相同的长度
train_seq_mat = sequence.pad_sequences(train_seq,maxlen=max_len)
train_seq_mat.shape

(1259, 30)

In [312]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_seq_mat,train_y, random_state=2,train_size=0.8)

In [313]:
X_train.shape

(1007, 30)

In [314]:
X_test.shape

(252, 30)

In [315]:
model = Sequential()
model.add(Embedding(max_words+1,60,input_length=max_len))
def build_model(num_layers,hidden_size,max_words=max_words,max_len=max_len,num_classes=2):
        '''
        hidden_size：LSTM隐藏层
        return_sequences:Boolen,return_sequences = True，
        则RNN层还可以返回每个样本的整个输出序列（每个样本每个时间步一个向量）。
        该输出的形状是（batch_size，timesteps，units）
        input_shape:输入训练数据的维度
        '''
        model = Sequential([
            Embedding(max_words+1,60,input_length=max_len),
            LSTM(hidden_size, return_sequences=True),
        ])
        for i in range(2,num_layers+1):
            if i == num_layers:
                model.add(LSTM(hidden_size, return_sequences=False))
            else:
                model.add(LSTM(hidden_size, return_sequences=True))
        model.add(Dense(128,activation="relu",name="FC1"))
        model.add(Dropout(0.5))
        model.add(Dense(num_classes,activation="sigmoid",name="FC2"))
        return model

In [316]:
model = build_model(3,64)
model.summary()

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_19 (Embedding)     (None, 30, 60)            300060    
_________________________________________________________________
lstm_25 (LSTM)               (None, 30, 64)            32000     
_________________________________________________________________
lstm_26 (LSTM)               (None, 30, 64)            33024     
_________________________________________________________________
lstm_27 (LSTM)               (None, 64)                33024     
_________________________________________________________________
FC1 (Dense)                  (None, 128)               8320      
_________________________________________________________________
dropout_10 (Dropout)         (None, 128)               0         
_________________________________________________________________
FC2 (Dense)                  (None, 2)               

In [317]:
model.compile(optimizer=Adam(0.001),
        loss='sparse_categorical_crossentropy',
        metrics=['sparse_categorical_accuracy'])

In [318]:
model_fit = model.fit(X_train,y_train,batch_size=1024,epochs=100,validation_data=(X_test,y_test))

Epoch 1/100


ValueError: in user code:

    E:\anaconda\envs\python36\lib\site-packages\tensorflow\python\keras\engine\training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    E:\anaconda\envs\python36\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    E:\anaconda\envs\python36\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    E:\anaconda\envs\python36\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    E:\anaconda\envs\python36\lib\site-packages\tensorflow\python\keras\engine\training.py:543 train_step  **
        self.compiled_metrics.update_state(y, y_pred, sample_weight)
    E:\anaconda\envs\python36\lib\site-packages\tensorflow\python\keras\engine\compile_utils.py:411 upd

In [215]:
model_fit = model.fit(train_seq_mat,train_y,batch_size=1024,epochs=300)

Epoch 1/300
2/2 [==============================] - 0s 4ms/step - loss: 0.6924 - sparse_categorical_accuracy: 0.5075
Epoch 2/300
2/2 [==============================] - 0s 6ms/step - loss: 0.6839 - sparse_categorical_accuracy: 0.8721
Epoch 3/300
2/2 [==============================] - 0s 6ms/step - loss: 0.6654 - sparse_categorical_accuracy: 0.9579
Epoch 4/300
2/2 [==============================] - 0s 7ms/step - loss: 0.6220 - sparse_categorical_accuracy: 0.9968
Epoch 5/300
2/2 [==============================] - 0s 6ms/step - loss: 0.5282 - sparse_categorical_accuracy: 1.0000
Epoch 6/300
2/2 [==============================] - 0s 6ms/step - loss: 0.4029 - sparse_categorical_accuracy: 1.0000
Epoch 7/300
2/2 [==============================] - 0s 7ms/step - loss: 0.3068 - sparse_categorical_accuracy: 0.9992
Epoch 8/300
2/2 [==============================] - 0s 6ms/step - loss: 0.2199 - sparse_categorical_accuracy: 1.0000
Epoch 9/300
2/2 [==============================] - 0s 6ms/step - loss: 0

2/2 [==============================] - 0s 6ms/step - loss: 1.6225e-04 - sparse_categorical_accuracy: 1.0000
Epoch 70/300
2/2 [==============================] - 0s 6ms/step - loss: 1.6590e-04 - sparse_categorical_accuracy: 1.0000
Epoch 71/300
2/2 [==============================] - 0s 6ms/step - loss: 1.7713e-04 - sparse_categorical_accuracy: 1.0000
Epoch 72/300
2/2 [==============================] - 0s 6ms/step - loss: 1.2626e-04 - sparse_categorical_accuracy: 1.0000
Epoch 73/300
2/2 [==============================] - 0s 6ms/step - loss: 1.6605e-04 - sparse_categorical_accuracy: 1.0000
Epoch 74/300
2/2 [==============================] - 0s 6ms/step - loss: 1.3486e-04 - sparse_categorical_accuracy: 1.0000
Epoch 75/300
2/2 [==============================] - 0s 6ms/step - loss: 1.6363e-04 - sparse_categorical_accuracy: 1.0000
Epoch 76/300
2/2 [==============================] - 0s 6ms/step - loss: 1.4981e-04 - sparse_categorical_accuracy: 1.0000
Epoch 77/300
2/2 [===========================

2/2 [==============================] - 0s 5ms/step - loss: 5.6856e-05 - sparse_categorical_accuracy: 1.0000
Epoch 137/300
2/2 [==============================] - 0s 7ms/step - loss: 5.8212e-05 - sparse_categorical_accuracy: 1.0000
Epoch 138/300
2/2 [==============================] - 0s 6ms/step - loss: 8.0688e-05 - sparse_categorical_accuracy: 1.0000
Epoch 139/300
2/2 [==============================] - 0s 6ms/step - loss: 6.8016e-05 - sparse_categorical_accuracy: 1.0000
Epoch 140/300
2/2 [==============================] - 0s 6ms/step - loss: 7.0328e-05 - sparse_categorical_accuracy: 1.0000
Epoch 141/300
2/2 [==============================] - 0s 6ms/step - loss: 6.0625e-05 - sparse_categorical_accuracy: 1.0000
Epoch 142/300
2/2 [==============================] - 0s 6ms/step - loss: 5.9052e-05 - sparse_categorical_accuracy: 1.0000
Epoch 143/300
2/2 [==============================] - 0s 6ms/step - loss: 6.3531e-05 - sparse_categorical_accuracy: 1.0000
Epoch 144/300
2/2 [===================

2/2 [==============================] - 0s 6ms/step - loss: 3.6450e-05 - sparse_categorical_accuracy: 1.0000
Epoch 204/300
2/2 [==============================] - 0s 6ms/step - loss: 4.9911e-05 - sparse_categorical_accuracy: 1.0000
Epoch 205/300
2/2 [==============================] - 0s 6ms/step - loss: 3.8131e-05 - sparse_categorical_accuracy: 1.0000
Epoch 206/300
2/2 [==============================] - 0s 6ms/step - loss: 3.0155e-05 - sparse_categorical_accuracy: 1.0000
Epoch 207/300
2/2 [==============================] - 0s 6ms/step - loss: 4.5848e-05 - sparse_categorical_accuracy: 1.0000
Epoch 208/300
2/2 [==============================] - 0s 6ms/step - loss: 4.8088e-05 - sparse_categorical_accuracy: 1.0000
Epoch 209/300
2/2 [==============================] - 0s 6ms/step - loss: 3.8640e-05 - sparse_categorical_accuracy: 1.0000
Epoch 210/300
2/2 [==============================] - 0s 6ms/step - loss: 5.1151e-05 - sparse_categorical_accuracy: 1.0000
Epoch 211/300
2/2 [===================

2/2 [==============================] - 0s 6ms/step - loss: 3.6988e-05 - sparse_categorical_accuracy: 1.0000
Epoch 271/300
2/2 [==============================] - 0s 6ms/step - loss: 1.9484e-05 - sparse_categorical_accuracy: 1.0000
Epoch 272/300
2/2 [==============================] - 0s 6ms/step - loss: 2.7099e-05 - sparse_categorical_accuracy: 1.0000
Epoch 273/300
2/2 [==============================] - 0s 6ms/step - loss: 1.3864e-05 - sparse_categorical_accuracy: 1.0000
Epoch 274/300
2/2 [==============================] - 0s 6ms/step - loss: 2.0918e-05 - sparse_categorical_accuracy: 1.0000
Epoch 275/300
2/2 [==============================] - 0s 7ms/step - loss: 2.1958e-05 - sparse_categorical_accuracy: 1.0000
Epoch 276/300
2/2 [==============================] - 0s 7ms/step - loss: 2.6259e-05 - sparse_categorical_accuracy: 1.0000
Epoch 277/300
2/2 [==============================] - 0s 6ms/step - loss: 1.8857e-05 - sparse_categorical_accuracy: 1.0000
Epoch 278/300
2/2 [===================

In [302]:
test_data = " ".join(list(tokenizer.cut('电流互感器二次负载阻抗的大小对互感器的准确度有很大影响。这是因为，如果电流互感器的')))

In [303]:
val_seq = tok.texts_to_sequences([test_data])
## 将每个序列调整为相同的长度
val_seq_mat = sequence.pad_sequences(val_seq,maxlen=max_len)
## 对验证集进行预测
val_pre = model.predict(val_seq_mat)

In [305]:
val_pre

array([[nan]], dtype=float32)

In [255]:
val_pre = model.predict(X_test)

In [256]:
val_pre.shape

(630, 2)

In [264]:
X_test

array([[  0,   0,   0, ..., 566,  31,   9],
       [  0,   0,   0, ..., 556,  31, 128],
       [110,   2, 431, ..., 315, 178, 123],
       ...,
       [  0,   0,   0, ..., 205, 936,  11],
       [129, 153,  91, ...,  24, 742,   2],
       [180,  23,  93, ...,  68, 404, 430]])

In [261]:
y_test

array([1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0,
       1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0,
       0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0,
       0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0,
       0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1,
       0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,

In [262]:
confm = metrics.confusion_matrix(y_pred = np.argmax(val_pre,axis=1),y_true=y_test)

In [263]:
confm

array([[305,   0],
       [  0, 325]], dtype=int64)